# http://bit.ly/wsd-2017

Наша первая задача - научиться делать Word Sense Disambiguation (WSD) при помощи семантических векторов.
Все необходимые библиотеки можно поставить при помощи команды

    pip install -U numpy scipy scikit-learn gensim rl_wsd_labeled
    
Скачайте word2vec модель по ссылке https://s3-us-west-2.amazonaws.com/models-tmp/sg_all_d256_m100.kv.zip и распакуйте архив.

Загрузим word2vec модель при помощи библиотеки [gensim](https://radimrehurek.com/gensim/models/word2vec.html):

In [9]:
from gensim.models import KeyedVectors
w2v_model = KeyedVectors.load('sg_all_d256_m100.kv')

Эта модель возвращает вектор по слову:

In [10]:
w2v_model['горшок']

array([ -9.92402732e-02,  -2.33064741e-01,   2.67826408e-01,
         2.18009073e-02,   1.52654126e-01,   6.13806583e-02,
         6.64153993e-02,   1.67186111e-01,   3.27731252e-01,
         1.79390356e-01,  -3.11140954e-01,   8.65942463e-02,
        -1.43792189e-03,   7.60920644e-02,  -2.56183594e-01,
         6.73418567e-02,   1.23391822e-02,  -2.60421127e-01,
        -6.60199463e-01,   7.46247619e-02,   5.24061732e-02,
         1.64030977e-02,   2.95160413e-01,  -3.17132776e-03,
         6.80546910e-02,  -5.99196404e-02,   1.16859920e-01,
         4.61518653e-02,   9.31969285e-02,   2.19365045e-01,
         2.76627213e-01,  -4.56820391e-02,  -2.70343926e-02,
        -1.62638649e-02,   4.17803228e-02,   8.82430002e-02,
         1.68497086e-01,   1.37637436e-01,   1.47348672e-01,
         3.52509655e-02,  -1.44522265e-01,   9.19886008e-02,
         1.16782561e-01,  -5.47234975e-02,   1.24640822e-01,
        -3.25096361e-02,  -4.08213139e-02,   2.04653814e-02,
        -2.44065523e-01,

In [11]:
w2v_model['горшок'].shape

(256,)

Она построена по лемматизированному корпусу, так что слова "горшка" в ней нет:

In [12]:
w2v_model['горшка']

KeyError: "word 'горшка' not in vocabulary"

Загрузим размеченные контексты для слова "горшок".

Мы занимаемся WSD, это supervised задача, так что нам нужны размеченные данные:

In [13]:
import rl_wsd_labeled

senses, contexts = rl_wsd_labeled.get_contexts(
    rl_wsd_labeled.contexts_filename('nouns', 'RuTenTen', 'горшок'))

In [14]:
senses

{'1': 'Округлый глиняный сосуд для приготовления пищи (печной горшок)',
 '2': 'Расширяющийся кверху сосуд с отверстием в дне (цветочный горшок)',
 '3': 'Ночной горшок'}

Каждый контекст имеет левую часть, слово для которого мы разметили значение, и правую часть:

In [15]:
len(contexts), contexts[:3]

(406,
 [(('телевизор, - ковер, , - музыкальный центр, - стол, - аквариум, - 3 шкафа, - цветы в',
    ' горшках',
    ', - мелкие аксессуары.'),
   '2'),
  (('перевалить в больший горшок, стараясь не повредить корни. Я выращиваю базилик в',
    ' горшках',
    ' объемом 0,7-1 литр. Этого ему вполне хватает. Можно брать горшки большего объема'),
   '2'),
  (('удобрениями. Весна это лучшее время для preszhdane горшечные растения в больший',
    ' горшок',
    ' со свежей почвой. Большинство из нас хорошо знают chitatelite эту практику, так'),
   '2')])

In [16]:
left, _, right = contexts[1][0]
left, right

('перевалить в больший горшок, стараясь не повредить корни. Я выращиваю базилик в',
 ' объемом 0,7-1 литр. Этого ему вполне хватает. Можно брать горшки большего объема')

Видно что контексты не лемматизованы, но в нашей модели есть только лемматизованые слова. Исправим это - загрузим mystem:

In [17]:
from pymystem3 import Mystem
mystem = Mystem()

In [18]:
mystem.lemmatize(left)

['перевалить',
 ' ',
 'в',
 ' ',
 'больший',
 ' ',
 'горшок',
 ', ',
 'стараться',
 ' ',
 'не',
 ' ',
 'повреждать',
 ' ',
 'корень',
 '. ',
 'я',
 ' ',
 'выращивать',
 ' ',
 'базилик',
 ' ',
 'в',
 '\n']

Нас будут интересовать только слова, отбросим знаки препинания и разделители:

In [19]:
import re

def tokenize(s):
    return [t for t in mystem.lemmatize(s)
            if re.match('\w+$', t)]

tokenize(left)

['перевалить',
 'в',
 'больший',
 'горшок',
 'стараться',
 'не',
 'повреждать',
 'корень',
 'я',
 'выращивать',
 'базилик',
 'в']

Теперь построим векторное представление контекста: возьмем среднее векторов всех слов контекста:

In [20]:
import numpy as np

def context_repr(context):
    left, _, right = context
    words = tokenize(left) + tokenize(right)
    return np.mean([w2v_model[w] for w in words if w in w2v_model],
                    axis=0)

context_repr(contexts[1][0])

array([ 0.00135083, -0.08833592,  0.02749542, -0.03430157,  0.04990597,
       -0.00351785, -0.0377895 ,  0.04325328,  0.03681576,  0.00777959,
       -0.09153348,  0.00741439, -0.00175712, -0.00361062, -0.06431929,
        0.03168813, -0.03431803, -0.04016968, -0.12728518, -0.05535114,
        0.05995908,  0.00678083,  0.01118196, -0.03073232,  0.00444479,
       -0.10487767, -0.02356375, -0.00680911,  0.05342769,  0.03441216,
        0.05997094, -0.0200747 ,  0.04236873, -0.03816266,  0.02926661,
       -0.00959332,  0.03278689,  0.01879168,  0.07646055, -0.01720432,
       -0.06004086, -0.02325079, -0.01187328,  0.04816246, -0.02895397,
       -0.02052972, -0.0224539 ,  0.03614068, -0.01640125, -0.00932809,
       -0.02643445,  0.10471622,  0.02175801,  0.10195923, -0.10238061,
       -0.01712064,  0.00831175,  0.00393068,  0.00358072,  0.02274694,
        0.0065462 , -0.05434006,  0.01133153, -0.03383037,  0.03999662,
        0.06762768, -0.0840156 ,  0.00743738, -0.05133355,  0.01

**Вопрос:** нужно ли использовать один и тот же лемматизатор для преобразования контекста и при построении word2vec модели?

Дальше подготовим данные для обучения:

In [21]:
from sklearn.neighbors import NearestCentroid
from sklearn.model_selection import cross_val_score

word = 'горшок'
senses, contexts = rl_wsd_labeled.get_contexts(
    rl_wsd_labeled.contexts_filename('nouns', 'RuTenTen', word))
xs = [ctx for ctx, _ in contexts]
ys = np.array([int(s) - 1 for _, s in contexts])

Построим векторные представления всех контекстов:

In [22]:
xs_vec = np.array([context_repr(ctx) for ctx in xs])

И обучим модель ``NearestCentroid`` используя кросс-валидацию:

In [23]:
scores = cross_val_score(NearestCentroid(), X=xs_vec, y=ys, cv=5)
print('Accuracy: {:.2f} ± {:.2f}'.format(np.mean(scores), 2 * np.std(scores)))

Accuracy: 0.81 ± 0.07


Наша метрика тут - точность. Но сколько можно получить, предсказывая самое частотное значение?

In [24]:
labels, counts = np.unique(ys, return_counts=True)
print('Most frequent sense baseline: {:.2f}'.format(
        np.mean(ys == labels[counts.argmax()])))

Most frequent sense baseline: 0.57


Теперь можно обучить классификатор на всех примерах и задать ему свой контекст для дизамбигуации:

In [25]:
clf = NearestCentroid()
clf.fit(xs_vec, ys)

NearestCentroid(metric='euclidean', shrink_threshold=None)

In [26]:
def predict(ctx):
    pred, = clf.predict([context_repr(ctx)])
    return senses[str(pred + 1)]

predict(('он полил цветы в', 'горшках', 'и выключил свет'))

'Расширяющийся кверху сосуд с отверстием в дне (цветочный горшок)'

In [27]:
predict(('он сел на', 'горшок', ''))

'Ночной горшок'

Задания - выбирать по вкусу:
1. Можете ли вы заставить классификатор ошибиться?
2. На каких тренировочных примерах он ошибается и почему?
3. А какая точность для других слов?
4. Как точность зависит от количества слов для обучения?
5. ``NearestCentroid`` не умеет работать с косинусной мерой близости, но известно что она работает лучше чем евклидова - сделайте классификатор который использует её и сравните качество.
6. Попробуйте использовать другие классификаторы из ``scikit-learn``, например ближайшие соседи, логистическую регрессию, другие. Какой классификатор работает лучше всех?
7. В представлении контекста (``context_repr``) мы учитываем все слова. Но все ли слова одинаково важны? Можете ли вы изменить эту функцию так, чтобы она давала более качественное предстваление контекста?
8. Какой будет результат если не использовать семантические вектора, а непосредственно слова?
9. Насколько важно качество word2vec модели?